In [6]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import toolkitEngineering
import importlib
importlib.reload(toolkitEngineering)

<module 'toolkitEngineering' from 'd:\\Projetos\\ETL-ML-Dengue\\src\\engineering\\toolkitEngineering.py'>

In [3]:
## Estabelecendo diretórios base
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_data = os.path.join(diretorio_base, 'data')

# Bronze
diretorio_bronze = os.path.join(diretorio_data, 'bronze')
diretorio_cnes_st_bronze = os.path.join(diretorio_bronze, 'cnes_st')
diretorio_cnes_st_bronze_csv = os.path.join(diretorio_cnes_st_bronze, 'csv')

# Silver
diretorio_silver = os.path.join(diretorio_data, 'silver')
diretorio_cnes_st_silver = os.path.join(diretorio_silver, 'cnes_st')
diretorio_cnes_st_silver_tsv = os.path.join(diretorio_cnes_st_silver, 'tsv')

## Criação de pastas para armazenar os arquivos
os.makedirs(os.path.join(diretorio_data), exist_ok=True)
os.makedirs(os.path.join(diretorio_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_cnes_st_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_cnes_st_silver_tsv), exist_ok=True)




In [5]:
arquivos_csv = [arquivo for arquivo in os.listdir(diretorio_cnes_st_bronze_csv) if arquivo.endswith("csv")]
arquivos_csv = arquivos_csv[-5:]
arquivos_tsv = [arquivo.rstrip("csv") + "tsv" for arquivo in arquivos_csv]
arquivos_tsv

['STSE1002.tsv',
 'STSP1001.tsv',
 'STSP1002.tsv',
 'STTO1001.tsv',
 'STTO1002.tsv']

In [ ]:
colunas_importantes = ['CNES', 'CODUFMUN', 'COMPETEN', 'VINC_SUS']

colunas_decodificar = {'VINC_SUS':{
    1: "Sim",
    0: "Não"
}}

column_map = []

columns_to_convert = []

In [ ]:
for arquivo_csv, arquivo_tsv in zip(arquivos_csv, arquivos_tsv):
    df = pd.read_csv(os.path.join(diretorio_cnes_st_bronze_csv, arquivo_csv), sep=',')

    for coluna in colunas_importantes:
        if coluna not in df.columns:
            df[coluna] = np.nan
            continue
    df = df[colunas_importantes]
    for coluna in colunas_decodificar:
        df[coluna] = df[coluna].apply(lambda x: toolkitEngineering.aplicar_decoficador(x, coluna, colunas_decodificar))
    df.replace("NA", np.nan, inplace=True)
    df.replace("", np.nan, inplace=True)
    df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
    df['NU_MES'] = df['DT_NOTIFIC'].apply(lambda x: x.split('-')[1].lstrip("0"))
    df = df.rename(columns=column_map)
    
    df.columns = df.columns.str.lower()
    
    for column in columns_to_convert:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int64')

    df.to_csv(os.path.join(diretorio_cnes_st_silver_tsv, arquivo_tsv), sep='\t', index=False)